In [3]:
with open("/home/tzy/Classification/datas/gl_dbs.json") as fp1:
    a = json.load(fp1)
with open("/home/tzy/Classification/datas/cihsy.json") as fp1:
    cihsy = json.load(fp1)

In [4]:
iplb = get_schema(a)

In [5]:
IP = [split_english(i) for i in iplb]
IP

[['Orders', 'Temp'],
 ['Register', 'Ex'],
 ['Order', 'Med'],
 ['Scalable', 'Field'],
 ['Archive', 'Record', 'Detail'],
 ['YBIC', 'Diagnosis'],
 ['Pat', 'Supplementary', 'Info'],
 ['Register', 'History'],
 ['VW', '', 'In', 'Fstpages'],
 ['Order', 'Operation'],
 ['Scalable', 'Template'],
 ['Archive', 'Template'],
 ['V', '', 'Ward', 'Bed'],
 ['Medical', 'Record', 'Operation', 'Situation'],
 ['Pat', 'Supplementary', 'Info', 'Data'],
 ['Room', 'Setting'],
 ['Order', 'Operation', 'Data'],
 ['Scalable', 'Template', 'Code', 'List'],
 ['Archive', 'Value', 'Field'],
 ['Treatment', '', 'PP'],
 ['Antibiotic', 'Usage', 'And', 'Purpose'],
 ['Third', 'Party', 'Import', 'Information'],
 ['Self', 'Recipe'],
 ['Order', 'Treatment', '', 'PP'],
 ['Order', 'Operation', 'Staff'],
 ['Scalable', 'Template', 'Dept', 'Code', 'List'],
 ['Patient', 'Clinical', 'Pathway'],
 ['Auto', 'Rate', 'Rule'],
 ['Register', 'Patient', 'Sign'],
 ['Self', 'Recipe', 'Data'],
 ['Order', 'Print', 'Log'],
 ['Scalable', 'Template',

In [6]:
syi = [mcfy(i,cihsy) for i in iplb]

In [7]:
from gensim.models import KeyedVectors

# 加载预训练 GloVe 模型
glove_file = 'glove.6B.100d.txt'
model = KeyedVectors.load_word2vec_format(glove_file, binary=False)

# 获取单词向量
vector = model['car']
print(vector)


[-0.1684   -0.53827   0.31155  -0.53218   0.26678  -0.13638   0.36621
  0.68383   0.77726   0.68049   0.69137   0.2103    0.091065  0.24845
 -0.16157   0.46291  -0.1503    0.2562   -0.1199    0.5913    1.0351
 -0.2052    0.30244  -0.34101   0.6326   -0.31603  -0.9959   -0.33583
  0.25161   0.10323   0.019611  0.54893  -0.33433   0.29617   0.41218
  0.4207    0.25775   0.12709   0.80269   0.61944   0.54316  -0.5941
  0.87551  -0.063686 -0.29117   0.61609   0.33376   0.14488  -0.039021
 -1.1849   -0.45951   0.15631  -0.50818   1.2357    0.30965  -1.958
 -1.1872    1.2027    2.1138    0.083629  0.54319   0.78883   0.35416
  0.87736   0.54007  -0.10454   0.075371 -0.45727  -0.27466   0.11838
 -0.49412  -0.61325   0.071519 -0.57665   0.21371   0.62137   1.4404
 -0.34033  -0.89958  -0.69605   0.74058   0.52105  -0.19224  -0.20366
 -0.22409  -0.3708   -0.34663  -0.86018  -0.89182  -0.43871   0.19424
  0.17073   0.43663  -0.11295  -0.51156   0.34186  -0.10274   0.39684
  1.734    -0.70787 ]


In [8]:
# 将文本转换为向量表示
vectors = []
i = 0
while i < len(IP):
    # 取每个文本中所有单词向量的平均值作为该文本的向量表示
    vec_mean = np.mean([model[w.lower()] for w in IP[i] if w.lower() in model], axis=0)
    if isinstance(vec_mean,np.ndarray):
        vectors.append(vec_mean)
        i+=1
    else:
        # print(IP[i])
        print(IP.pop(i))
        print(syi.pop(i))
        print(iplb.pop(i))

['Fstpage']
首页__
Fstpage
['GLJB', '', 'GZHCAPPLY']
未知____工伤申请__
GLJB_GZHCAPPLY


/opt/miniconda3/envs/neo4j/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/opt/miniconda3/envs/neo4j/lib/python3.8/site-packages/numpy/core/_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [17]:
for i in vectors:
    try:
        n = len(i)
        if n!=100:
            print(n)
    except:
        print(type(i))

In [128]:

# 使用 KMeans 聚类算法进行聚类
k = 114  # 聚类数目
kmeans = KMeans(n_clusters=k, random_state=0).fit(vectors)

# 输出聚类结果
labels = kmeans.labels_
with open('km_res.txt','w') as f:
    for i in range(k):
        f.write(f'Cluster {i+1}:\n')
        for j, text in enumerate(zip(syi,iplb)):
            if labels[j] == i:
                f.write(f'{text}\n')

/opt/miniconda3/envs/neo4j/lib/python3.8/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [120]:
from sklearn.metrics import silhouette_score
# 计算轮廓系数
s = silhouette_score(vectors, labels, metric='euclidean')
print("轮廓系数：", s)

from sklearn.metrics import davies_bouldin_score
# 计算DI值
di = davies_bouldin_score(vectors, labels)
print("davies_bouldin_score: ", di)

轮廓系数： 0.12106423
davies_bouldin_score:  1.5199252297297727


In [121]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score
# 计算DI值
intra_dist = []
for i in range(kmeans.n_clusters):
    cluster_i = np.array(vectors)[labels == i]
    dist = pairwise_distances(cluster_i)
    ddi = [d for dd in dist for d in dd if d!=0.0]
    if ddi:
        intra_dist.append(min(ddi))
max_inter_dist = pairwise_distances(kmeans.cluster_centers_).max()
di = min(intra_dist) / max_inter_dist

# 计算DBI值
centroids = kmeans.cluster_centers_
intra_dists = []
for i in range(kmeans.n_clusters):
    cluster_i = np.array(vectors)[labels == i]
    centroid_i = centroids[i]
    intra_dists.append(np.mean(pairwise_distances(cluster_i, centroid_i.reshape(1, -1))))

dbi = 0
for i in range(kmeans.n_clusters):
    max_dij = 0
    for j in range(kmeans.n_clusters):
        if i != j:
            dist = pairwise_distances(centroids[i].reshape(1, -1), centroids[j].reshape(1, -1))
            dij = (intra_dists[i] + intra_dists[j]) / dist
            max_dij = max(max_dij, dij)
    dbi += max_dij
dbi /= kmeans.n_clusters

print("DI: ", di)
print("DBI: ", dbi)

DI:  6.7699957969676244e-09
DBI:  [[1.51992521]]


In [113]:
from sklearn.cluster import DBSCAN
clustering = DBSCAN(eps= 2,min_samples= 2).fit(vectors)
# 输出聚类结果
labels = clustering.labels_
max(labels)+2

114

In [114]:
for i in range(max(labels)+2):
    print(f'Cluster {i+1}:')
    for j, text in enumerate(zip(syi,iplb)):
        if labels[j] == i-1:
            print(f'  {text}')

Cluster 1:
  ('医嘱__临时__', 'OrdersTemp')
  ('登记__扩展__', 'RegisterEx')
  ('可扩展的__字段__', 'ScalableField')
  ('登记__历史__', 'RegisterHistory')
  ('未知____在……里__首页__', 'VW_InFstpages')
  ('可扩展的__模板__', 'ScalableTemplate')
  ('归档__模板__', 'ArchiveTemplate')
  ('未知____病房__床位__', 'V_WardBed')
  ('房间__设置__', 'RoomSetting')
  ('归档__值__字段__', 'ArchiveValueField')
  ('抗生素__用法__和__目的__', 'AntibioticUsageAndPurpose')
  ('个人 自助__处方__', 'SelfRecipe')
  ('自动__速率__规则__', 'AutoRateRule')
  ('登记__患者__体征 签名__', 'RegisterPatientSign')
  ('个人 自助__处方__数据__', 'SelfRecipeData')
  ('临床路径（Clinical Pathway）__阶段__图表__', 'CPPhaseDiagram')
  ('结算__', 'Settlement')
  ('医嘱 订单__申请__', 'OrderRequest')
  ('医生__授权__', 'DoctorAuthorization')
  ('医嘱 订单__过渡__', 'OrderTransition')
  ('处方__动态__表格 表单__', 'RecipeDynamicForm')
  ('图表__数据__触发器__', 'ChartDataTrigger')
  ('医嘱 订单__动态__表格 表单__', 'OrderDynamicForm')
  ('打印__历史__', 'PrintHistory')
  ('处方__生病__注意__', 'RecipeSickNote')
  ('打印__模板__', 'PrintTemplate')
  ('患者__固定__项目__', 'PatFix

In [118]:
from sklearn.metrics import silhouette_score
# 计算轮廓系数
s = silhouette_score(vectors, labels, metric='euclidean')

print("轮廓系数：", s)
from sklearn.metrics import davies_bouldin_score
# 计算DI值
di = davies_bouldin_score(vectors, labels)
print("davies_bouldin_score: ", di)

轮廓系数： 0.019106306
davies_bouldin_score:  1.6155114349970465


In [116]:
np.unique(labels)

array([ -1,   0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,
        12,  13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,
        25,  26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,
        38,  39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,
        51,  52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,
        64,  65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,
        77,  78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,
        90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102,
       103, 104, 105, 106, 107, 108, 109, 110, 111, 112])

In [122]:
from sklearn.metrics import pairwise_distances
from sklearn.metrics import davies_bouldin_score

# 计算DI值
intra_dist = []
for i in np.unique(labels):
    if i == -1:
        continue
    cluster_i = np.array(vectors)[labels == i]
    dist = pairwise_distances(cluster_i)
    ddi = [d for dd in dist for d in dd if d!=0.0]
    if ddi:
        intra_dist.append(min(ddi))
max_inter_dist = pairwise_distances(np.array(vectors)[labels != -1], metric='euclidean').max()
di = min(intra_dist) / max_inter_dist

# 计算DBI值
intra_dists = []
for i in np.unique(labels):
    if i == -1:
        continue
    cluster_i = np.array(vectors)[labels == i]
    centroid_i = np.mean(cluster_i, axis=0)
    intra_dists.append(np.mean(pairwise_distances(cluster_i, centroid_i.reshape(1, -1))))

dbi = 0
for i in np.unique(labels):
    if i == -1:
        continue
    max_dij = 0
    for j in np.unique(labels):
        if i == j or j == -1:
            continue
        # if i == j:
        #     continue
        dist = pairwise_distances(np.mean(np.array(vectors)[labels == i], axis=0).reshape(1, -1), np.mean(np.array(vectors)[labels == j], axis=0).reshape(1, -1))
        dij = (intra_dists[i] + intra_dists[j]) / dist
        max_dij = max(max_dij, dij)
    dbi += max_dij
dbi /= len(np.unique(labels)) - 1

print("DI: ", di)
print("DBI: ", dbi)

DI:  6.694499e-09
DBI:  [[1.5333759]]
